<h1> Engineer Holidayness Feature </h1>

In [2]:
from pathlib import Path
import sys
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg
from src import eda_utils
from src import plotting_utils
from src import config as cfg
from src import filter_group_aggr_utils as fga

from typing import Tuple
import matplotlib.pyplot as plt
import numpy as np
import math


<h2> Build the Baden-Württemberg DataFrame </h2>

We load the DataFrames from the `../data/processed` Folder 

In [14]:
DATA_PROCESSED = Path("../data/processed")

# --- Load Parquet files ---
OPSD_60min_DE_LU_df = pd.read_parquet(DATA_PROCESSED / "OPSD_60min_de_lu_df.parquet")
DE_hol_df  = pd.read_parquet(DATA_PROCESSED / "DE_hol_df.parquet")

# --- Quick check ---
print("Loaded DataFrames:")
print(f"OPSD_60min_DE_LU_df.shape: {OPSD_60min_de_lu_df.shape}")
print(f"DE_hol_df.shape: {DE_hol_df.shape}")


Loaded DataFrames:
OPSD_60min_DE_LU_df.shape: (50401, 45)
DE_hol_df.shape: (7670, 37)


We make copies of the DataFrames, with features relevant to **Baden-Württemberg**  only. 

In [7]:
DE_BW_hol_df = DE_hol_df[[ "local_date", "local_start", "local_end",	"utc_start",	"utc_end",	"DE_BW_hol",	"DE_BW_school_free"]].copy()
DE_BW_hol_df

,local_date,local_start,local_end,utc_start,utc_end,DE_BW_hol,DE_BW_school_free
0,2015-01-01,2015-01-01 00:00:00+01:00,2015-01-02 00:00:00+01:00,2014-12-31 23:00:00+00:00,2015-01-01 23:00:00+00:00,1,1
1,2015-01-02,2015-01-02 00:00:00+01:00,2015-01-03 00:00:00+01:00,2015-01-01 23:00:00+00:00,2015-01-02 23:00:00+00:00,0,1
2,2015-01-03,2015-01-03 00:00:00+01:00,2015-01-04 00:00:00+01:00,2015-01-02 23:00:00+00:00,2015-01-03 23:00:00+00:00,0,1
3,2015-01-04,2015-01-04 00:00:00+01:00,2015-01-05 00:00:00+01:00,2015-01-03 23:00:00+00:00,2015-01-04 23:00:00+00:00,0,1
4,2015-01-05,2015-01-05 00:00:00+01:00,2015-01-06 00:00:00+01:00,2015-01-04 23:00:00+00:00,2015-01-05 23:00:00+00:00,0,1
...,...,...,...,...,...,...,...
7665,2035-12-27,2035-12-27 00:00:00+01:00,2035-12-28 00:00:00+01:00,2035-12-26 23:00:00+00:00,2035-12-27 23:00:00+00:00,0,0
7666,2035-12-28,2035-12-28 00:00:00+01:00,2035-12-29 00:00:00+01:00,2035-12-27 23:00:00+00:00,2035-12-28 23:00:00+00:00,0,0
7667,2035-12-29,2035-12-29 00:00:00+01:00,2035-12-30 00:00:00+01:00,2035-12-28 23:00:00+00:00,2035-12-29 23:00:00+00:00,0,0
7668,2035-12-30,2035-12-30 00:00:00+01:00,2035-12-31 00:00:00+01:00,2035-12-29 23:00:00+00:00,2035-12-30 23:00:00+00:00,0,0


In [15]:
BW_cols = [col for col in OPSD_60min_de_lu_df.columns if "transnetbw" in col or  "timestamp" in col]
print(BW_cols)

['utc_timestamp', 'cet_cest_timestamp', 'DE_transnetbw_load_actual_entsoe_transparency', 'DE_transnetbw_load_forecast_entsoe_transparency', 'DE_transnetbw_solar_generation_actual', 'DE_transnetbw_wind_onshore_generation_actual']


In [17]:
OPSD_60min_BW_df =  OPSD_60min_DE_LU_df[bw_cols]
print(OPSD_60min_BW_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50401 entries, 0 to 50400
Data columns (total 6 columns):
 #   Column                                           Non-Null Count  Dtype                        
---  ------                                           --------------  -----                        
 0   utc_timestamp                                    50401 non-null  datetime64[ns, UTC]          
 1   cet_cest_timestamp                               50401 non-null  datetime64[ns, Europe/Berlin]
 2   DE_transnetbw_load_actual_entsoe_transparency    50400 non-null  float64                      
 3   DE_transnetbw_load_forecast_entsoe_transparency  50376 non-null  float64                      
 4   DE_transnetbw_solar_generation_actual            50386 non-null  float64                      
 5   DE_transnetbw_wind_onshore_generation_actual     50400 non-null  float64                      
dtypes: datetime64[ns, Europe/Berlin](1), datetime64[ns, UTC](1), float64(4)
memory usage: 2